In [58]:
from sklearn.datasets import load_files
from collections import Counter
from nltk.tokenize import word_tokenize
import pandas as pd
import re
pd.options.mode.chained_assignment = None

In [59]:
class AutoCorrect:
    def from_file(self,file):
        vocablist=[]
        with open(file,encoding='utf-8-sig',mode='r') as file:
            lines=file.readlines()
            for line in lines:
                vocablist +=[word for word in re.findall(r'\w+',line) if len(word)>1]
        vocab_counter=Counter(vocablist)
        totalwords=sum(vocab_counter.values())
        self.vocab=set(vocablist)
        self.word_prob={word:vocab_counter[word]/totalwords for word in vocab_counter.keys()}

    def from_dir(self,dir):
        dataset=load_files(dir,encoding='utf-8-sig')
        vocablist=[]
        for doc in dataset['data']:
            vocablist +=[word for word in re.findall(r'\w+',doc) if len(word)>1]
        vocab_counter=Counter(vocablist)
        totalwords=sum(vocab_counter.values())
        self.vocab=set(vocablist)
        self.word_prob={word:vocab_counter[word]/totalwords for word in vocab_counter.keys()}


    def _level_one_edits(self,word):
        letters=u'ابتةثجحخدذرزسشصضطظعغفقكلمنهويءآأؤإئ'
        splits=[(word[:part],word[part:]) for part in range(0,len(word))]
        deletes=[part1+part2[1:] for part1,part2 in splits if part2]
        swaps=[part1+part2[0]+part2[1]+part2[2:] for part1,part2 in splits if len(part2) > 1]
        replaces=[part1+c+part2[1:] for part1,part2 in splits if part2 for c in letters]
        inserts=[part1+c+part2 for part1,part2 in splits for c in letters]
        return set(deletes+swaps+replaces+inserts)

    def _level_two_edits(self,word):
        return set(e2 for e1 in self._level_one_edits(word) for e2 in self._level_one_edits(e1))

    def check(self,word):
        if word in self.vocab:
            return True
        candidates =list(self._level_one_edits(word))+list(self._level_two_edits(word))+[word]
        valid_candidates = [w for w in candidates if w in self.vocab]
        if len(valid_candidates) > 0 :
            return sorted([(c, self.word_prob[c]) for c in valid_candidates], key=lambda tup: tup[1], reverse=True)
        else:
            return False

    def correct(self,word):
        if word in self.vocab:
            return word
        candidates =list(self._level_one_edits(word))+list(self._level_two_edits(word))+[word]
        valid_candidates = [w for w in candidates if w in self.vocab]
        if len(valid_candidates) > 0 :
            return sorted([(c, self.word_prob[c]) for c in valid_candidates], key=lambda tup: tup[1], reverse=True)[0][0]
        else:
            return word


In [60]:
autoCorrect=AutoCorrect()

In [61]:
autoCorrect.from_dir('Khaleej-2004')

In [62]:
df=pd.read_csv('Cleaned_data.csv')
df.head()

,Author,Book_Name,Book_Category,Book_url,Publishing_house,Book_size_MB,Book_pages,Quotation
0,الكاتب محمود سالم,لغز القبر الملكي,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,52.0,NaN
1,الكاتب محمود سالم,لغز الكوخ المحترق,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,56.0,NaN
2,الكاتب محمود سالم,لغز اللص الشبح,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,48.0,NaN
3,الكاتب محمود سالم,لغز المدينة العائمة,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,54.0,NaN
4,الكاتب محمود سالم,لغز المنزل رقم 98,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,48.0,NaN


In [63]:
df[~(df["Quotation"].isna())]

,Author,Book_Name,Book_Category,Book_url,Publishing_house,Book_size_MB,Book_pages,Quotation
7,الكاتب هارون يحيى,التضحية عند الحيوان,كتب الأدب العربي-كتب متنوعة,https://www.arab-books.com/books/%d9%83%d8%aa%...,عدنان أوكطار,5.00,160.0,ترون بأنفسكم نماذج من السلوك عند بعض الحيوانات...
8,الكاتب هارون يحيى,معجزة الذرة,كتب تطوير الذات-كتب متنوعة,https://www.arab-books.com/books/%d9%83%d8%aa%...,عدنان أوكطار,3.41,136.0,إن هذا الكتاب يبين وبالدليل العلمي أن الذي خلق...
9,الكاتب هارون يحيى,لا تتجاهل,كتب الأدب العربي,https://www.arab-books.com/books/%d9%83%d8%aa%...,عدنان أوكطار,3.00,100.0,هذه الحقيقة بالإضافة إلى حقائق كونية أخرى يستع...
10,الكاتب هارون يحيى,الحياة في سبيل الله,كتب اسلامية,https://www.arab-books.com/books/%d9%83%d8%aa%...,عدنان أوكطار,2.11,100.0,أما المنافقون فهم في سبيل الفوز بما قال عنه ال...
11,الكاتب هارون يحيى,العظمة في كل مكان,كتب الأدب العربي,https://www.arab-books.com/books/%d9%83%d8%aa%...,عدنان أوكطار,5.30,140.0,ولو سال عن كيفية التي يتحرك بها كل شيء هذا الك...
...,...,...,...,...,...,...,...,...
1182,الكاتب الياس ابو شبكة,القيثارة لإلياس أبو شبكة,كتب الأدب العربي-كتب روايات عربية,https://www.arab-books.com/books/%d9%83%d8%aa%...,هنداوي,5.00,198.0,لقد فتك ت بك يد أثيمة في بلاد الغربة بعيد ا عن...
1183,الكاتب الياس ابو شبكة,المريض الصامت,كتب الأدب العربي-كتب روايات عربية,https://www.arab-books.com/books/%d9%83%d8%aa%...,هنداوي,1.00,30.0,إن ما تقرأ في هذه الصفحات لهو حقيقة أليمة نطقت...
1185,الكاتب الياس ابو شبكة,العمال الصالحون,كتب الأدب العربي-كتب روايات عربية,https://www.arab-books.com/books/%d9%83%d8%aa%...,هنداوي,1.00,80.0,كان لبيب راغب ولد ا في العاشرة من سنيه جميل ال...
1186,الكاتب الياس ابو شبكة,الألحان ومقتطفات من غلواء,كتب الأدب العربي-كتب روايات عربية,https://www.arab-books.com/books/%d9%83%d8%aa%...,هنداوي,13.00,22.0,إلى الحصاد جنى الجهاد قلب البلاد يحيا بنا هيا ...


In [86]:
#Auto Correct data
book_names=[]
book_Quotation=[]
for record in df["Book_Name"]:
    record=re.sub("\d","",record)
    words=word_tokenize(record)
    words_new=[]
    for word in words:
        if len(word) < 2:
            words_new.append(autoCorrect.correct(word))
        else:
            words_new.append(word)
    book_names.append(' '.join(words_new))

for record in df[~(df["Quotation"].isna())]["Quotation"]:
    record=re.sub("\d","",record)
    words=word_tokenize(record)
    words_new=[]
    for word in words:
        if len(word) < 2:
            words_new.append(autoCorrect.correct(word))
        else:
            words_new.append(word)
    book_Quotation.append(' '.join(words_new))

In [87]:
df["Book_Name"]=book_names
df[~(df["Quotation"].isna())]["Quotation"]=book_Quotation

In [88]:
df["Book_Category"]=df["Book_Category"].str.split('-')
df=df.explode("Book_Category")
df

,Author,Book_Name,Book_Category,Book_url,Publishing_house,Book_size_MB,Book_pages,Quotation
0,الكاتب محمود سالم,لغز القبر الملكي,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.00,52.0,NaN
1,الكاتب محمود سالم,لغز الكوخ المحترق,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.00,56.0,NaN
2,الكاتب محمود سالم,لغز اللص الشبح,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.00,48.0,NaN
3,الكاتب محمود سالم,لغز المدينة العائمة,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.00,54.0,NaN
4,الكاتب محمود سالم,لغز المنزل رقم,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.00,48.0,NaN
...,...,...,...,...,...,...,...,...
5975,الكاتب فرانز كافكا,رواية التحول فرانتس كافكا,كتب الروايات العالمية المترجمة,https://www.arab-books.com/books/%d8%b1%d9%88%...,منشورات الجمل,1.64,106.0,NaN
5976,الشيخ عبدالوهاب خلاف,احكام الاحوال الشخصية في الشريعة الاسلامية,كتب أصول الفقه,https://www.arab-books.com/books/%d9%83%d8%aa%...,دار الكتب العلمية,14.00,289.0,NaN
5977,الله عز وجل,Corán traducido al español في القران نسخة اسبانية,القرآن الكريم,https://www.arab-books.com/books/coran-traduci...,محمد رسول الله,26.00,1088.0,NaN
5978,الله عز وجل,القران الكريم باللغة الانجليزية -Quran with En...,القرآن الكريم,https://www.arab-books.com/books/%d8%a7%d9%84%...,محمد رسول الله,30.00,978.0,NaN


In [89]:
#split data to two models one with Quotation and with not
df1=df[df["Quotation"].isna()]
df2=df[~(df["Quotation"].isna())]

In [90]:
df1=df1[df.columns[:-1]]
df1.head()

,Author,Book_Name,Book_Category,Book_url,Publishing_house,Book_size_MB,Book_pages
0,الكاتب محمود سالم,لغز القبر الملكي,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,52.0
1,الكاتب محمود سالم,لغز الكوخ المحترق,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,56.0
2,الكاتب محمود سالم,لغز اللص الشبح,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,48.0
3,الكاتب محمود سالم,لغز المدينة العائمة,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,54.0
4,الكاتب محمود سالم,لغز المنزل رقم,روايات بوليسية,https://www.arab-books.com/books/%d9%83%d8%aa%...,مؤسسة هنداوي,2.0,48.0


In [91]:
df1['Book_Category'].value_counts()

كتب اسلامية                  1495
كتب روايات عربية              855
روايات بوليسية                621
كتب الأدب العربي              417
العقيدة                       387
                             ... 
كتب شعر مترجم                   1
كتب أدب السجون                  1
الرياضات المائية والتجديف       1
علوم الحاسب                     1
كتب الجريمة                     1
Name: Book_Category, Length: 78, dtype: int64

In [92]:
df2["Book_Category"].value_counts()

كتب الأدب العربي                   114
كتب روايات عربية                    75
كتب الروايات العالمية المترجمة      50
كتب اسلامية                         35
كتب التاريخ                         23
كتب التاريخ القديم                  20
كتب الفلسفة والمنطق                 13
الثقافة العامة                       2
كتب متنوعة                           2
كتب أساطير                           1
كتب المسرحيات العالمية المترجمة      1
كتب العلوم الإنسانية                 1
كتب التنمية الذاتية                  1
كتب علم النفس                        1
روايات بوليسية                       1
كتب تطوير الذات                      1
كتب التاريخ الحديث                   1
الأدب                                1
أعلام وشخصيات                        1
كتب طبية                             1
العلوم السياسية والاستراتيجية        1
كتب أدب السجون                       1
كتب و دواوين الشعر العربي            1
Name: Book_Category, dtype: int64

In [145]:
#prepare the training for model 1
x1_test=df1[df1["Book_Category"].isna()]["Book_Name"].tolist()
x1_train=df1[~(df1["Book_Category"].isna())]["Book_Name"].tolist()
y1_train=df1[~(df1["Book_Category"].isna())]["Book_Category"].tolist()

In [141]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [149]:
pipe=Pipeline([("Vector",TfidfVectorizer(encoding='utf-8-sig',lowercase=False)),("Model",LogisticRegression(max_iter=2000))])

In [150]:
pipe.fit(x1_train,y1_train)

Pipeline(steps=[('Vector',
                 TfidfVectorizer(encoding='utf-8-sig', lowercase=False)),
                ('Model', LogisticRegression(max_iter=2000))])

In [151]:
pipe.score(x1_train,y1_train)

0.5891994731450315